<a href="https://colab.research.google.com/github/ambgeo/geoquantificacao/blob/main/05_App_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install streamlit streamlit-folium pyngrok branca

In [ ]:
import ee
try:
    ee.Authenticate()
except Exception as e:
    print("Se já autenticou nesta máquina, ignore:", e)

# ajuste o project se necessário
ee.Initialize(project='ee-scriptsambgeo')
print("GEE inicializado.")


In [ ]:
%%writefile app.py
import streamlit as st
import geemap.foliumap as geemap     # mapa folium via geemap
from streamlit_folium import st_folium
from folium.plugins import Draw       # ferramenta de desenho
import ee
import pandas as pd
import plotly.express as px

# ============= Config da página =============
st.set_page_config(page_title="MapBiomas Coleção 10 — Brasil", layout="wide")
st.title("🗺️ MapBiomas Brasil — Coleção 10 (Coverage v2)")
st.caption("Asset: projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2")

# ============= Earth Engine =============
try:
    ee.Authenticate()
except Exception as e:
    print("Se já autenticou nesta máquina, ignore:", e)

# ajuste o project se necessário
ee.Initialize(project='ee-scriptsambgeo')
print("GEE inicializado.")

# ============= Parâmetros (UI) =============
YEARS = list(range(1985, 2024))
st.sidebar.header("Parâmetros")
ano = st.sidebar.slider("Ano", min_value=YEARS[0], max_value=YEARS[-1], value=YEARS[-1], step=1)
basemap = st.sidebar.selectbox("Basemap", ["CartoDB.Positron", "OpenStreetMap", "HYBRID", "SATELLITE", "TERRAIN"], index=0)

# ============= Classes e paleta (cores oficiais) =============
CLASS_INFO = {
    # 1. Floresta
    1:  ("Floresta", "#1f8d49"),
    3:  ("Formação Florestal", "#1f8d49"),
    4:  ("Formação Savânica",  "#7dc975"),
    5:  ("Mangue",             "#04381d"),
    6:  ("Floresta Alagável",  "#007785"),
    49: ("Restinga Arbórea",   "#02d659"),
    # 2. Vegetação Herbácea e Arbustiva
    10: ("Veg. Herb./Arbust.", "#d6bc74"),
    11: ("Campo Alagado/Pant.", "#519799"),
    12: ("Formação Campestre", "#d6bc74"),
    32: ("Apicum",             "#fc8114"),
    29: ("Aflor. Rochoso",     "#ffaa5f"),
    50: ("Restinga Herbácea",  "#ad5100"),
    # 3. Agropecuária
    14: ("Agropecuária",       "#ffefc3"),
    15: ("Pastagem",           "#edde8e"),
    18: ("Agricultura",        "#E974ED"),
    19: ("Lavoura Temporária", "#C27BA0"),
    39: ("Soja",               "#f5b3c8"),
    20: ("Cana",               "#db7093"),
    40: ("Arroz",              "#c71585"),
    62: ("Algodão (beta)",     "#ff69b4"),
    41: ("Outras Lavouras Temp", "#f54ca9"),
    36: ("Lavoura Perene",     "#d082de"),
    46: ("Café",               "#d68fe2"),
    47: ("Citrus",             "#9932cc"),
    35: ("Dendê",              "#9065d0"),
    48: ("Outras Lavouras Per.", "#e6ccff"),
    9:  ("Silvicultura",       "#7a5900"),
    21: ("Mosaico de Usos",    "#ffefc3"),
    # 4. Área não Vegetada
    22: ("Área não Vegetada",  "#d4271e"),
    23: ("Praia/Duna/Areal",   "#ffa07a"),
    24: ("Área Urbanizada",    "#d4271e"),
    30: ("Mineração",          "#9c0027"),
    75: ("Fotovoltaica (beta)","#c12100"),
    25: ("Outras áreas não veg", "#db4d4f"),
    # 5. Água
    26: ("Corpo d'água",       "#2532e4"),
    33: ("Rio/Lago/Oceano",    "#2532e4"),
    31: ("Aquicultura",        "#091077"),
    # 6. Não observado
    27: ("Não observado",      "#ffffff"),
}
CLASS_CODES = sorted(CLASS_INFO.keys())
PALETTE = [CLASS_INFO[c][1] for c in CLASS_CODES]

# ============= Carrega o asset e prepara visualização (remap p/ paleta) =============
asset_id = "projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2"
banda = f"classification_{ano}"

try:
    lulc_raw = ee.Image(asset_id).select(banda)  # códigos originais (para a estatística)
except Exception as e:
    st.error(f"Erro ao abrir banda {banda}: {e}")
    st.stop()

# Remapeia códigos não contíguos -> índices 1..N apenas para visualizar com a paleta
lulc_vis = lulc_raw.remap(CLASS_CODES, list(range(1, len(CLASS_CODES)+1)))
vis_params = {"min": 1, "max": len(CLASS_CODES), "palette": PALETTE}

# ============= Mapa com geemap + ferramenta de desenho =============
m = geemap.Map(center=[-14.235, -51.9253], zoom=4, control_scale=True)
try:
    m.add_basemap(basemap)
except Exception:
    m.add_basemap("CartoDB.Positron")

# Camada de uso e cobertura (visual)
m.addLayer(lulc_vis, vis_params, f"MapBiomas Col10 - {ano}")

# Legenda
legend_dict = {f"{c} — {CLASS_INFO[c][0]}": CLASS_INFO[c][1] for c in CLASS_CODES}
m.add_legend(title=f"MapBiomas {ano} — Coleção 10", legend_dict=legend_dict)

# Ferramenta de desenho (polígono/retângulo)
draw = Draw(
    export=False, position='topleft',
    draw_options={
        "polygon": True, "rectangle": True,
        "polyline": False, "circle": False,
        "circlemarker": False, "marker": False
    },
    edit_options={"edit": True, "remove": True}
)
draw.add_to(m)

# ⚠️ Para capturar o desenho, use st_folium (to_streamlit não retorna os GeoJSONs)
out = st_folium(m, height=600, use_container_width=True)

st.caption("Dica: desenhe um polígono/retângulo e abaixo clique em **Gerar gráfico**.")

# ============= Botão para gerar a estatística =============
if st.button("Gerar gráfico por classe na ROI"):
    # 1) Recupera o último desenho (GeoJSON)
    roi_geojson = None
    if out and out.get("last_active_drawing"):
        roi_geojson = out["last_active_drawing"]
    elif out and out.get("all_drawings"):
        if len(out["all_drawings"]) > 0:
            roi_geojson = out["all_drawings"][-1]

    if roi_geojson is None:
        st.warning("Nenhuma geometria desenhada. Desenhe uma ROI no mapa e tente novamente.")
        st.stop()

    # 2) Converte GeoJSON -> ee.Geometry (aceita Feature ou Geometry)
    geom = roi_geojson.get("geometry", roi_geojson)  # se vier como Feature
    try:
        roi = ee.Geometry(geom)
    except Exception as e:
        st.error(f"Falha ao interpretar a ROI desenhada: {e}")
        st.stop()

    # 3) Estatística de área por classe (km²) usando group reducer
    #    - pixelArea (m²) -> km²
    area_km2 = ee.Image.pixelArea().divide(1e6).rename("area_km2")
    img_for_group = area_km2.addBands(lulc_raw.rename("class"))

    # groupField=1 => agrupa pela 2ª banda (class)
    reduced = img_for_group.reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1, groupName="class"),
        geometry=roi,
        scale=30,         # resolução MapBiomas ~30m
        maxPixels=1e13,
        tileScale=4
    )

    groups = reduced.get("groups")
    try:
        groups_py = groups.getInfo()  # lista de dicts: {'class': code, 'sum': area_km2}
    except Exception as e:
        st.error(f"Não foi possível obter a estatística: {e}")
        st.stop()

    # 4) Monta DataFrame com nome e cor oficiais
    rows = []
    codes_present = set()
    if isinstance(groups_py, list):
        for g in groups_py:
            code = int(g.get("class"))
            area = float(g.get("sum", 0.0))
            if code in CLASS_INFO:
                nome, cor = CLASS_INFO[code]
                rows.append({"codigo": code, "classe": nome, "cor": cor, "area_km2": area})
                codes_present.add(code)

    # Se alguma classe não apareceu, opcionalmente mantém com zero (comente se não quiser)
    for code in CLASS_CODES:
        if code not in codes_present:
            nome, cor = CLASS_INFO[code]
            rows.append({"codigo": code, "classe": nome, "cor": cor, "area_km2": 0.0})

    df = pd.DataFrame(rows).sort_values("area_km2", ascending=False).reset_index(drop=True)

    # 5) Gráfico de barras com as cores oficiais
    color_map = {r["classe"]: r["cor"] for _, r in df.iterrows()}
    fig = px.bar(
        df[df["area_km2"] > 0],              # mostra só classes presentes (>0)
        x="classe", y="area_km2",
        color="classe",
        color_discrete_map=color_map,
        labels={"classe": "Classe", "area_km2": "Área (km²)"},
        title=f"Área por classe — {ano} (ROI desenhada)"
    )
    fig.update_layout(showlegend=False, xaxis_tickangle=-30)

    st.plotly_chart(fig, use_container_width=True)
    st.dataframe(df, use_container_width=True)

    # 6) Download CSV
    st.download_button(
        "Baixar CSV",
        data=df.to_csv(index=False).encode("utf-8"),
        file_name=f"mapbiomas_col10_{ano}_area_por_classe.csv",
        mime="text/csv"
    )


In [ ]:
# ⬇️ baixa o binário do cloudflared e dá permissão de execução
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared
!chmod +x /content/cloudflared

# ▶️ inicia o Streamlit em background na porta 8501
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 >/content/streamlit.log 2>&1 &

# 🌐 abre o túnel público (a célula exibe 2 URLs https; use a que contém trycloudflare.com)
!/content/cloudflared tunnel --url http://localhost:8501 --no-autoupdate
